In [ ]:
#confusion, precision, accuracy, recall을 한번에 보여주는 함수
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0: .4f}, 정밀도: {1: .4f}, 재현율: {2: .4f}'.format(accuracy, precision, recall))

In [ ]:
# precision/recall trade-off (이진 분류에서)
# predict는 예측값을 반환
# predict_prob()은 0일 때의 확률이 얼마고, 1일때의 확률이 얼마인지 반환
# 분류 결정값이 0.5이었음. predict_prob은 0일때와 1일때의 확률중 0.5가 큰것을 반환
pred_proba = lr_clf.predict_proba(X_test)
pred = lr_clf.predict(X_test)

# Binarizer 활용하여 임계값 조정
from sklearn.preprocessing import Binarizer
# 임의의 X 행렬
binarizer = Binarizer(threshold=1.1)
binarizer.fit_transform(X)

# 여러 개의 분류 결정 임계값을 변경하면서 Binarizer을 이용하여 예측값 변환
# thresholds가 올라갈수록 precision이 올라가는 것을 볼 수 있음.
thresholds = [.4, .45, .5, .55, .6]
def get_eval_by_threshold(y_test, pred_proba_c1, thresholds):
    # thresholds list 객체내의 값을 차례로 iteration하면서 Evaluation 수행.
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold = custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        get_clf_eval(y_test, custom_predict)
        
# precision과 recall을 그리는 함수
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

def precision_recall_curve_plot(y_test, pred_proba_c1,):
    # threshold ndarray와 threshold에 따른 정밀도, 재현율 ndarray 추출
    precisions, recalls, thresholds = precision_recall_curve(y_test, pred_proba_c1)
    
    # X축을 threshold값으로, y축은 정밀도 재현율 값으로 각각 plot 수행. 정밀도는 점선으로 표시
    plt.figure(figsize=(8,6))
    threshold_boundary = thresholds.shape[0]
    plt.plot(thresholds, precisions[0:threshold_boundary], linestyle='--', label='precision')
    plt.plot(thresholds, recalls[0:threshold_boundary], label='recall')
    
    # threshold 값 x 축의 scale을 0.1 단위로 변경
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    
    # x축, y축 label과 legend, 그리고 grid 설정
    plt.xlabel('Threshold value')
    plt.ylabel('Precision and Recall value')
    plt.legend()
    plt.grid()
    plt.show()
# 실제값 데이터 셋과 레이블 값이 1일 때의 예측 확률을 precision_recall_curve 인자로 입력
precision_recall_curve_plot(y_test, lr_clf.predict_proba(X_test)[:,1])

In [ ]:
# f1 score 구하기
from sklearn.metrics import f1_score
f1 = f1_score(y_test, pred)
f1

# roc_curve
def roc_curve_plot(y_test, pred_proba_c1):
    # 임계값에 따른 FPR, TPR 값을 반환 받음
    fprs, tprs, thresholds = roc_curve(y_test, pred_proba_c1)
    
    # ROC Curve를 plot 곡선으로 그림
    plt.plot(fprs, tprs, label='ROC')
    # 가운데 대각선을 그림
    plt.plot([0,1], [1,0], 'k--', label='Random')
    
    # FPR X축의 Scale을 0.1 단위로 변경, X,Y 축명 설정 등
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1), 2))
    plt.xlim(0,1)
    plt.ylim(0,1)
    plt.xlabel('FPR(1-Sensitivity)')
    plt.ylabel('TPR(Recall)')
    plt.legend()
    plt.show()
    
roc_curve_plot(y_test, lr_clf.predict_proba(X_test)[:, 1])

# area under score 값 (ROC)
from sklearn.metrics import roc_auc_score
pred_proba = lr_clf.pred_proba(X_test)[:, 1]
roc_score = roc_auc_score(y_test, pred_proba)
roc_score

## 모든 평가 점수를 다 불러오는 함수
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuarcy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    # ROC-AUC 추가
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0: .4f}, 정밀도: {1: .4f}, 재현율: {2: .4f},\
           F1: {3: .4f}, AUC: {4: .4f}'.format(accuarcy, precision, recall, f1, roc_auc))